In [ ]:
!python3 -m pip install ../input/nflimpactdetectron/detectron2/detectron2 --no-index --find-links ../input/nflimpactdetectron/wheelz/wheelz

In [ ]:
import pickle
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog
from detectron2.engine import DefaultPredictor,DefaultTrainer
from detectron2.model_zoo import model_zoo
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import MetadataCatalog, DatasetCatalog, build_detection_train_loader, build_detection_test_loader, \
    DatasetMapper
from detectron2.structures import BoxMode, Instances, Boxes
from detectron2.modeling import build_model
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.structures import Instances

from detectron2.modeling.roi_heads.roi_heads import StandardROIHeads
from detectron2.modeling import ROI_HEADS_REGISTRY

from typing import Dict, List, Optional, Tuple

from detectron2.utils.visualizer import Visualizer

import cv2
import torch
from glob import glob
from copy import deepcopy 
from tqdm import tqdm

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import sys
sys.path.append('../input/footballhelmetdetector')
from model import get_predictor
# \, new_model_cfg
from train_helmet_detector import NflImpactTrainer

In [ ]:
!ls extracted_test_video

In [ ]:
from pathlib import Path

ROOT = Path.cwd().parent
INPUT = ROOT / "input"
OUTPUT = ROOT / "working"
DATA = INPUT / "nfl-impact-detection"
TRAIN_VIDEOS = DATA / "train"
TEST_VIDEOS = DATA / "test"

# 
EXTRACTED_TEST_VIDEOS = OUTPUT / "extracted_test_video"

WORK = ROOT / "working"

# # save frames out of /kaggle/working/ because of the HDD limitation
TMP = ROOT / "tmp"
TRAIN_EXTRACTED_FRAMES = TMP / "nfl-impact-detection-train-frames"

try:
    TRAIN_EXTRACTED_FRAMES.mkdir(parents=True)
except:
    print('file exist')

In [ ]:
def mk_images(video_name, video_labels, video_dir, out_dir, only_with_impact=True):
    video_path=f"{video_dir}/{video_name}"
    video_name = os.path.basename(video_path)
    vidcap = cv2.VideoCapture(video_path)
    if only_with_impact:
        boxes_all = video_labels.query("video == @video_name")
        print(video_path, boxes_all[boxes_all.impact == 1.0].shape[0])
    else:
        print(video_path)
    frame = 0
    while True:
        it_worked, img = vidcap.read()
        if not it_worked:
            break
        frame += 1
        if only_with_impact:
            boxes = video_labels.query("video == @video_name and frame == @frame")
            boxes_with_impact = boxes[boxes.impact == 1.0]
            if boxes_with_impact.shape[0] == 0:
                continue
        img_name = f"{video_name}_frame{frame}"
        image_path = f'{out_dir}//{video_name}'.replace('.mp4',f'_{str(frame).zfill(3)}.png')
        _ = cv2.imwrite(image_path, img)

In [ ]:
# !mkdir test
# !cp ../input/nfl-impact-detection/test/57906_000718_Endzone.mp4 ./test/57906_000718_Endzone.mp4

# Extracted image from test video into DATA_ROOT_PATH

In [ ]:
SEED = 42
!rm -r helmet_crop_test
out_dir = EXTRACTED_TEST_VIDEOS

if not os.path.exists(out_dir):
    !mkdir -p $out_dir
    video_dir = '/kaggle/input/nfl-impact-detection/test'
    uniq_video = [path.split('/')[-1] for path in glob(f'{video_dir}/*.mp4')]
    for video_name in uniq_video:
        mk_images(video_name, pd.DataFrame(), video_dir, out_dir, only_with_impact=False)
else:
    print('can not extracted video to frame')

In [ ]:
def new_model_cfg():
    cfg = get_cfg()
    model = "COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"
    cfg.merge_from_file(model_zoo.get_config_file(model))
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(model)
    cfg.MODEL.MASK_ON = False
    cfg.INPUT.RANDOM_FLIP = "none"
    cfg.OUTPUT_DIR = "output"

    cfg.DATALOADER.NUM_WORKERS = 2
    cfg.SOLVER.IMS_PER_BATCH = 4
    cfg.SOLVER.CHECKPOINT_PERIOD = 2000
    cfg.SOLVER.STEPS = (21000, 50000)
    cfg.SOLVER.MAX_ITER = 200000
    cfg.SOLVER.BASE_LR = 0.001
    cfg.TEST.EVAL_PERIOD = 2000
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
    cfg.DATALOADER.FILTER_EMPTY_ANNOTATIONS = True
#     cfg.MODEL.DEVICE = 'cpu'
    return cfg


def get_predictor(path = "../input/nflimpactdetectron/helmet_detector_model_0093999.pth"):
    classes = ['helmet']
    MetadataCatalog.get("nflimpact").set(thing_classes=classes)
    cfg = new_model_cfg()
    cfg.MODEL.WEIGHTS = path
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(classes)
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
    return DefaultPredictor(cfg)


# predictor = build_model(cfg)

# images = ImageList.from_tensors()  # preprocessed input tensor

# model = build_model(cfg)
# model.eval()
# features = model.backbone(images.tensor)
# proposals, _ = model.proposal_generator(images, features)
# instances, _ = model.roi_heads(images, features, proposals)
# mask_features = [features[f] for f in model.roi_heads.in_features]
# mask_features = model.roi_heads.mask_pooler(mask_features, [x.pred_boxes for x in instances])

# predictor = get_predictor('../input/run-detectron-helmet-detector/output/model_0001999.pth') # e2e


predictor = get_predictor('../input/nflimpactdetectron/helmet_detector_model_0093999.pth') # box predictor only

In [ ]:
def bboxes2xywh(df):
    # row to multiple rows     
    df = df[df.bboxes.apply(lambda x: len(x)!=0)]
    df = df.explode('bboxes').reset_index(drop=True)
    
#     df['bboxes'] = df['bboxes'].apply(lambda x: x.tensor.to('cpu').numpy()[0])
    
    df[['x','y','w','h']] = pd.DataFrame(df.bboxes.tolist(), index= df.index)

    df['w'] = df['w'] - df['x']
    df['h'] = df['h'] - df['y']
    df.x = df.x.round()
    df.y = df.y.round()
    df.w = df.w.round()
    df.h = df.h.round()
    
    df = df.rename(columns={"x": "left", "w":"width", "y": "top", "h":"height"})
    df = df.drop(columns=['bboxes'])
    return df


# Get bboxes of helmet using 2nd stage detectron2

In [ ]:

# path = '../input/nfl-impact-detection-train-frames/57583_000082_Endzone'
dic_list = []

for path in os.listdir(EXTRACTED_TEST_VIDEOS):
    gameKey, playID, view, frame = path.split('_')
    video = '_'.join([gameKey, playID, view]) + '.mp4'
    gameKey = int(gameKey)
    playID = int(playID)
    frame = int(frame[:-4])
    im = cv2.imread(f"{EXTRACTED_TEST_VIDEOS}/{path}")
#     print(im)

    model_output = predictor(im)
    
    model_output = model_output["instances"].to("cpu")
    filter_mask = model_output.scores > 0.8
    ni = Instances(model_output.image_size, **{
        "scores": model_output.scores[filter_mask],
        "pred_boxes": model_output.pred_boxes[filter_mask],
        "pred_classes": model_output.pred_classes[filter_mask]
    })
    
#     crop bigger bboxes from output of object detection

    bboxes = ni.get('pred_boxes').tensor.numpy()

    dic = {'gameKey':gameKey, 'playID':playID, 'view':view, 'video':video, 'frame':frame, 'bboxes':bboxes,
#           'video':video
          }
    dic_list.append(dic)


In [ ]:
with open('dic_list.pickle', 'wb') as handle:
    pickle.dump(dic_list, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
df = pd.DataFrame(dic_list)
df = bboxes2xywh(df)

In [ ]:
df.head()

# Careful convert btw xywh and xyxy here

In [ ]:
# Call this for the output of detectron

def mk_crop_imgs(df,split='train', input_folder = "../input/nfl-impact-detection-train-frames/", is_nested_folder = True):
    max_w, max_h = 1280, 720
    def _convert_tl_br(magnitude, x1,y1,w_box,h_box):
        assert magnitude >0, 'increase size <0'
        x2 = x1 + w_box
        y2 = y1 + h_box
        w_box = abs(x2 - x1)*magnitude
        assert w_box >0, 'w_box <0'
        h_box = abs(y2 - y1)*magnitude
        assert h_box >0, 'h_box <0'

        x1 = x1 - w_box if (x1 - w_box) >0 else 0
        y1 = y1 - h_box if (y1 - h_box) >0 else 0

        x2 = x2 + w_box if (x2 + w_box) <max_w else max_w
        y2 = y2 + h_box if (y2 + h_box) <max_h else max_h

        return int(x1),int(y1),int(x2),int(y2)
    
    def _convert_tl_br_fixed_size(size, x,y,w,h):
        h_target,w_target = size
        # if original box is smaller than target box   
        if(w<= w_target and h<= h_target):
            center_x = x+ np.floor(w/2)
            center_y = y+ np.floor(h/2)

            x1 = center_x - np.floor(w_target/2) if center_x - np.floor(w_target/2) >0 else 0
            y1 = center_y - np.floor(h_target/2) if center_y - np.floor(h_target/2) >0 else 0

            x2 = center_x + np.floor(w_target/2) if center_x + np.floor(w_target/2) < max_w else max_w
            y2 = center_y + np.floor(h_target/2) if center_y + np.floor(h_target/2) < max_h else max_h
        # if original box is bigger than target box   
        else:
            x1 = x
            y1 = y
            x2 = x + w
            y2 = y + h
        return int(x1),int(y1),int(x2),int(y2)


    def crop_img(row):
        # setup path
        
        input_name = row['image_name']
        
        if (not('label' in row.keys()) and not('impact' in row.keys()) ):
#             Need to change this to distinguish btw small bboxes, use index of df ???
            output_name = input_name
        else:
            playerLabel = row['label']
            impact = int(row['impact'])
            output_name = f"{input_name[:-4]}_{playerLabel}_{impact}.png"
        
        
        path = f"helmet_crop_{split}/"
        
        
        #         convert coordinate to be magnitude**2 time bigger
        #x1,y1,x2,y2 = _convert_tl_br(1, *row[['x','y','w','h']])
        # convert bboxes to fix size
        x1,y1,x2,y2 =_convert_tl_br_fixed_size((100,100), *row[['x','y','w','h']])
        
        # final path to input image       
        if is_nested_folder:
            nested_folder = row['video'].replace('.mp4', '')
            big_img_path = os.path.join(input_folder,nested_folder, row['image_name'])
        else:
            big_img_path = os.path.join(input_folder, row['image_name'])
#         print(big_img_path)
        img = cv2.imread(big_img_path)

        assert img.shape.__len__() == 3 , 'image is incorrect'
        crop_img = img[y1:y2, x1:x2]
        
        
        
        w_box = abs(x2 - x1)
        h_box = abs(y2 - y1)
        if(w_box < 100 or h_box<100):
            try:
                resized_image = cv2.resize(crop_img, (100, 100)) 
                cv2.imwrite(os.path.join(path,output_name), resized_image)
    #             print('write img ok')
            except:
                print(x1,y1,x2,y2)
                print(f"crop_img = {crop_img.shape}")
                print(f"resize_img = {resized_image.shape}.")
                print(f"smt wrong with {output_name} at w_box < 100 or h_box<100")
        else:
            try:
                
                cv2.imwrite(os.path.join(path, output_name), crop_img)
    #             print('write img ok')
            except:
                print(x1,y1,x2,y2)
                print(f"crop_img = {crop_img.shape}")
                print(f"smt wrong with {output_name}")
    df = df.reset_index(drop=True)
    
    for index,row in df.iterrows():
        crop_img(row)

In [ ]:
def convert_name_df(df):
    def convert_name(input_row):
    #     print(row)
        row = input_row.copy()
        input_name = row.image_name
        if (not('label' in row.keys()) and not('impact' in row.keys()) ):
            return input_name
        else:
            return f"{input_name[:-4]}_{playerLabel}_{int(impact)}.png" 
    df['image_name'] = df.apply(lambda x: x['video'].replace('.mp4', '') + '_' + str(x['frame']).zfill(3) + '.png', axis=1)
    df['image_name'] = df.apply(lambda x: convert_name(x), axis=1)
#     df['label'] = '666'
    return df

# Crop image to 100x100 size from video frame (image) 

In [ ]:
df = convert_name_df(df)
df = df.rename(columns={"left": "x", "width":"w", "top": "y", "height":"h"})

In [ ]:
!ls

In [ ]:

mk_crop_imgs(df, input_folder= EXTRACTED_TEST_VIDEOS, split='test', is_nested_folder = False) #this will output to ./helmet_crop_test
# resnet on this df

In [ ]:
class ImageDatasetBoxes(Dataset) :
    def __init__(self, df=None, transform=None, img_size=(100,100), mask= False, input_folder="../input/resnet18-on-bboxes/", name="train") :
        self.df = df
        self.img_size = img_size
        self.transform = transform
        self.mask = mask
        
#         print(f"name={name}")
        
        if name == 'train':
            self.input_folder = os.path.join(input_folder,'helmet_crop_train')
        elif name == 'test':
            self.input_folder = os.path.join(input_folder,'helmet_crop_test')
            
    def __len__(self) :
        return len(self.df)
    
    # fix this for inference
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.input_folder, row.image_name)
        image = cv2.imread(img_path)
        if self.transform:
            transformed_img = train_transform(image=image)['image']

        if name == "train":
            if int(row.impact) == 0:
                label = torch.tensor([1,0])
            elif int(row.impact) == 1:
                label = torch.tensor([0,1])
            else:
                print(row.impact)
                print(type(row.impact))
                assert False, 'incorrect label type'
            return transformed_img, label
        
        elif name == "test":
            return transformed_img, row
        else:
            assert name !== 'test' and name !== 'train', 

In [ ]:
transform = A.Compose(
    [
        Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
        ),
        ToTensor()
    ]
)

# TODO: include input folder 
test_dataset = ImageDatasetBoxes(submit_df,transform= transform, input_folder= EXTRACTED_TEST_VIDEOS,= name = "test")
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [ ]:
model = torch.load('resnet_epoch_no_mask4.pt')['model']
predicts = []
output_rows = []

model.eval()
with torch.no_grad():            
#     with torch.set_grad_enabled(False):
    for images, rows in tqdm(test_loader) :
        images = images.to(device)             
        l = l.to(device)

        out = model(image)
        out_ = out.argmax(1)
        out_ = out_.to('cpu').numpy().tolist()
        
        df_rows = pd.DataFrame(rows)
        df_rows['impact'] = out_
        
        output_rows.append(df_rows)

final_submit_df = pd.DataFrame(output_rows)

In [ ]:
def mk_crop_imgs(df,split='train'):
    max_w, max_h = 1280, 720
    def _convert_tl_br(magnitude, x1,y1,w_box,h_box):
        assert magnitude >0, 'increase size <0'
        x2 = x1 + w_box
        y2 = y1 + h_box
        w_box = abs(x2 - x1)*magnitude
        assert w_box >0, 'w_box <0'
        h_box = abs(y2 - y1)*magnitude
        assert h_box >0, 'h_box <0'

        x1 = x1 - w_box if (x1 - w_box) >0 else 0
        y1 = y1 - h_box if (y1 - h_box) >0 else 0

        x2 = x2 + w_box if (x2 + w_box) <max_w else max_w
        y2 = y2 + h_box if (y2 + h_box) <max_h else max_h

        return int(x1),int(y1),int(x2),int(y2)
    
    def _convert_tl_br_fixed_size(size, x,y,w,h):
        h_target,w_target = size
        # if original box is smaller than target box   
        if(w<= w_target and h<= h_target):
            center_x = x+ np.floor(w/2)
            center_y = y+ np.floor(h/2)

            x1 = center_x - np.floor(w_target/2) if center_x - np.floor(w_target/2) >0 else 0
            y1 = center_y - np.floor(h_target/2) if center_y - np.floor(h_target/2) >0 else 0

            x2 = center_x + np.floor(w_target/2) if center_x + np.floor(w_target/2) < max_w else max_w
            y2 = center_y + np.floor(h_target/2) if center_y + np.floor(h_target/2) < max_h else max_h
        # if original box is bigger than target box   
        else:
            x1 = x
            y1 = y
            x2 = x + w
            y2 = y + h
        return int(x1),int(y1),int(x2),int(y2)


    def crop_img(row):
        #         setup path
        input_folder = "../input/nfl-impact-detection-train-frames/"
        input_name = row['image_name']
        playerLabel = row['label']
        path = f"helmet_crop_{split}/"
        
        #         convert coordinate to be magnitude**2 time bigger
        #x1,y1,x2,y2 = _convert_tl_br(1, *row[['x','y','w','h']])
        # convert bboxes to fix size
        x1,y1,x2,y2 =_convert_tl_br_fixed_size((100,100), *row[['x','y','w','h']])
        
        input_path = row['video'].replace('.mp4', '')
        
        # final path to input image         
        the_path = os.path.join(input_folder,input_path, row['image_name'])
#         print(f"{the_path}")
        img = cv2.imread(the_path)
        
        assert img.shape.__len__() == 3, 'incorrect image'
        crop_img = img[y1:y2, x1:x2]
        
        output_name = f"{input_name[:-4]}_{playerLabel}_{int(row['impact'])}.png"
        
        w_box = abs(x2 - x1)
        h_box = abs(y2 - y1)
        if(w_box < 100 or h_box<100):
            try:
                resized_image = cv2.resize(crop_img, (100, 100)) 
                cv2.imwrite(os.path.join(path,output_name), resized_image)
    #             print('write img ok')
            except:
                print(x1,y1,x2,y2)
                print(f"crop_img = {crop_img.shape}")
                print(f"resize_img = {resized_image.shape}.")
                print(f"smt wrong with {output_name} at w_box < 100 or h_box<100")
        else:
            try:
                
                cv2.imwrite(os.path.join(path,output_name), crop_img)
    #             print('write img ok')
            except:
                print(x1,y1,x2,y2)
                print(f"crop_img = {crop_img.shape}")
                print(f"smt wrong with {output_name}")
    df = df.reset_index(drop=True)
    for index,row in tqdm(df.iterrows()):
#         print(index)
        crop_img(row)
#         break

In [ ]:
import nflimpact
env = nflimpact.make_env()
env.predict(final_submit_df) # df is a pandas dataframe of your entire submission file

In [ ]:
!ls ../input/utils-to-prepare-different-type-of-dataset